In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
from IPython.display import display, Image, clear_output
import time

robot = Robot()
camera = Camera.instance()

In [ ]:
import dt_apriltags

detector = dt_apriltags.Detector()

while True:
    clear_output(wait = True)
    image = np.array(camera.value)
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    detections = detector.detect(gray_frame)

    # Draw bounding boxes and tag IDs
    for detection in detections:
        corners = detection.corners
        corners = [(int(x), int(y)) for (x, y) in corners]

        # Draw the bounding box around the tag
        cv2.polylines(image, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)

        # Draw the tag ID
        tag_id = detection.tag_id
        cv2.putText(image, f"ID: {tag_id}", (int(corners[0][0]), int(corners[0][1])-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    display(Image(data=bgr8_to_jpeg(image)))
    time.sleep(0.05)
